In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import tree
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [37]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/train_dataset_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/test_dataset_test.csv")
df_solution = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Новосиб/sample_solution.csv")

In [38]:
df_sub = pd.merge(df_solution, df_test, on="id", how="left")
df_sub = df_sub.drop(["Class", "id"], axis = 1)

## Рассмотрим датасет по ближе

In [39]:
df_train['Class'] = df_train['Class'].map({0:0, 1:1, 3:2, 4:3, 5:4, 64:5})
df_solution['Class'] = df_solution['Class'].map({0:0, 1:1, 3:2, 4:3, 5:4, 64:5})

In [40]:
df_train

,id,Easting,Northing,Height,Reflectance,Class
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,2
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0
3,2454459,431680.4542,6.032343e+06,69.1892,-11.21,0
4,4608150,431720.0914,6.032288e+06,67.3252,-9.10,0
...,...,...,...,...,...,...
4222089,2498809,431721.6917,6.032297e+06,69.2220,-9.21,0
4222090,146939,431684.7126,6.032323e+06,67.5003,-9.52,0
4222091,5641497,431735.0991,6.032293e+06,68.9625,-10.30,0
4222092,3833813,431702.1040,6.032318e+06,69.2873,-9.94,0


Пострим на распределение данных

## Выделим выборки

In [41]:
df_train = df_train.fillna(0)

In [42]:
X = df_train.drop(["Class", "id"], axis = 1)
y = df_train[["Class"]]

In [43]:
ss = StandardScaler()

In [52]:
params = { 'max_depth': [10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7575)

In [45]:
#neigh = RadiusNeighborsClassifier(radius=3.0)
#neigh.fit(X, y)
#predNeighbor = neigh.predict(X_test)

## Обучение модели

In [46]:
#clf = RandomForestClassifier(n_estimators = 100, max_depth = 3, class_weight="balanced", max_features="log2", min_weight_fraction_leaf=0.2, random_state=7575)

In [47]:
#clf.fit(X_train, y_train)

In [48]:
#clfXGBoost = XGBClassifier(max_depth=4)

In [ ]:
clfXGBoost = XGBClassifier(tree_method="gpu_hist")
clf = GridSearchCV(estimator=clfXGBoost, 
                   param_grid=params,
                   scoring='recall', 
                   verbose=0)
clf.fit(X, y)
print("Best parameters:", clf.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

In [49]:
clfXGBoost.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(max_depth=10, objective='multi:softprob', tree_method='gpu_hist')

In [51]:
pred = clfXGBoost.predict(df_sub)
_id = df_solution["id"]
sub = pd.DataFrame({'id': _id , 'Class': pred})
sub['Class'] = sub['Class'].map({0.0:0, 1.0:1, 2.0:3, 3.0:4, 4.0:5, 5.0:64})
sub.to_csv('sub.csv', index=False)